In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 20)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [2]:
matches = pd.read_csv("data/euro_soccer/Matches.csv")
teams = pd.read_csv("data/euro_soccer/Team.csv")
teams_attr = pd.read_csv("data/euro_soccer/Team_Attributes.csv")
players = pd.read_csv("data/euro_soccer/Player.csv")
players_attr = pd.read_csv("data/euro_soccer/Player_Attributes.csv")
countries = pd.read_csv("data/euro_soccer/Country.csv")
leagues = pd.read_csv("data/euro_soccer/League.csv")

In [41]:
sample = matches.dropna().iloc[0:2, :]\
.drop(["Unnamed: 0", "id"], axis=1)\
.drop(matches.columns[86:], axis=1)\
.drop(["match_api_id", ], axis=1)

In [42]:
sample.head()

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
3248,1729,1729,2012/2013,1,2012-08-18 00:00:00,1228229,9825,8472,0,0,...,175947.0,33468.0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>52</comment><event...
3249,1729,1729,2012/2013,1,2012-08-20 00:00:00,1228230,8668,10260,1,0,...,160713.0,113836.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>32</comment><event...


In [59]:
sample_teams = sample.merge(teams.rename(columns={"team_api_id":"home_team_api_id"}), \
                            on=["home_team_api_id"], how="inner")

In [88]:
sample_teams_drop = sample_teams\
.drop(sample_teams.columns[84:87], axis=1)\
.drop(sample_teams.columns[0:4], axis=1)\
.drop(["team_short_name", "match_api_id", "home_team_api_id"], axis=1)\
.rename(columns={"team_long_name":"home_team"})
# sample_teams.head().T

In [89]:
sample_teams_drop = sample_teams_drop.merge(teams.rename(columns={"team_api_id":"away_team_api_id"}), \
                            on=["away_team_api_id"], how="inner")

In [90]:
sample_teams_drop = sample_teams_drop\
.drop(sample_teams_drop.columns[79:82], axis=1)\
.drop(["team_short_name"], axis=1)\
.rename(columns={"team_long_name":"away_team"})

In [91]:
sample_teams_drop.head().T

,0,1
date,2012-08-18 00:00:00,2012-08-20 00:00:00
away_team_api_id,8472,10260
home_team_goal,0,1
away_team_goal,0,0
home_player_X1,1,1
home_player_X2,2,2
home_player_X3,4,4
home_player_X4,6,6
home_player_X5,8,8
home_player_X6,3,2
